In [2]:
#Imports
import psycopg2
import pandas as pd

#### Prep CSV as Dataframe ####

#Function to remove $
def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return(float(x.replace('$', '')))
    return(x)

df = pd.read_csv('salesdata.csv')
df['amount'] = df['amount'].apply(clean_currency)

#Drop null values
df = df.dropna()

#Export dataframe to cleaned csv
df.to_csv('salesinfo.csv', index=False)

#### Connect to Database ####

#Retrieve Username and Password 
user_file = open('../user_file.txt', 'r') 
username = user_file.readline() 
password = user_file.readline() 

#Setup Connection
connection = psycopg2.connect(
    host = 'arkatechture-assignment.chzhrd21j2xp.us-east-2.rds.amazonaws.com',
    port = 5432,
    user = str(username),
    password = str(password),
    database='arkatechture_assignment'
    )
cursor=connection.cursor()
cursor=connection.cursor()

sql = "DROP TABLE salesdata"

cursor.execute(sql)


#Create Table with SQL Commands 
cursor.execute("""CREATE TABLE salesdata(
sales_person text,
sales_department text,
amount decimal,
quantity integer,
street text,
city text,
state text,
item_id text,
date DATE)""")

connection.commit()

#Copy in data from csv
with open('salesinfo.csv', 'r') as row:
    next(row)
    cursor.copy_from(row, 'salesdata', sep=',')
    
connection.commit()
